In [1]:
!pip install ffmpeg-python pillow
!git clone https://github.com/soCzech/TransNetV2.git
%cd TransNetV2/inference

Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 362 (delta 70), reused 70 (delta 70), pack-reused 278
Receiving objects: 100% (362/362), 95.27 KiB | 1.73 MiB/s, done.
Resolving deltas: 100% (210/210), done.
/kaggle/working/TransNetV2/inference


In [2]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg
import torch
import numpy as np
from tqdm import tqdm
from transnetv2 import TransNetV2

2024-08-09 09:59:27.537098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 09:59:27.537229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 09:59:27.689870: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model = TransNetV2()

[TransNetV2] Using weights from /kaggle/working/TransNetV2/inference/transnetv2-weights/.


In [11]:
def extract_keyframes(video_path, output_dir):
    _, single_frame_predictions, _ = model.predict_video(video_path)
    
    scenes = model.predictions_to_scenes(single_frame_predictions)
    
    video_name = os.path.basename(video_path).split('.')[0]
    
    output_dir = output_dir + video_name + "/"
    
    if not os.path.exists(output_dir):
            os.makedirs(output_dir)
    
    with open(f"{output_dir}{video_name}_frames_stamp.json", 'w') as f:
        json.dump(scenes.tolist(), f)
        
    cap = cv2.VideoCapture(video_path)
    
    frames = []
    
    for idx, scene in enumerate(scenes):
        start_frame, end_frame = scene
        middle_frame = (start_frame + end_frame) // 2
        for frame_index in [start_frame, middle_frame, end_frame]:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
            ret, frame = cap.read()
            frames.append((frame_index, frame))
            
    frames_output_dir = output_dir + video_name + "_keyframes/"
            
    if not os.path.exists(frames_output_dir):
            os.makedirs(frames_output_dir)
            
    for (frame_index, frame) in frames:
        output_path = os.path.join(frames_output_dir, f"frame_{frame_index}.jpg")
        cv2.imwrite(output_path, frame)
    return video_name, output_dir

In [5]:
import subprocess

def extract_audio(video_path, audio_path):
    command = f"ffmpeg -i '{video_path}' -ab 160k -ac 2 -ar 44100 -vn '{audio_path}'"
    subprocess.call(command, shell=True)

In [10]:
import zipfile
import shutil
def zip_folder(folder_path, output_zip_path):
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)
    shutil.rmtree(folder_path)

In [7]:
def extract_data_and_pack(video_path, output_dir):
    video_name, output_dir = extract_keyframes(video_path, output_dir)
    extract_audio(video_path, output_dir + video_name + "_audio.mp3")
    zip_folder(output_dir[:len(output_dir) - 1], output_dir[:len(output_dir) - 1] + ".zip")
    

In [12]:
import time
start = time.time()
extract_data_and_pack("/kaggle/input/c00-vidieo/AIC_VideoB1v1/Videos_L01/video/L01_V001.mp4","/kaggle/working/")
end = time.time()
print(end-start)

[TransNetV2] Extracting frames from /kaggle/input/c00-vidieo/AIC_VideoB1v1/Videos_L01/video/L01_V001.mp4
[TransNetV2] Processing video frames 27714/27714


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

207.44793820381165
